In [5]:
%pip install langchain openai langchain-community langchain-openai beautifulsoup4 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.4 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [9]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
urls = ['https://n.news.naver.com/article/015/0005181264?sid=101', 'https://n.news.naver.com/article/015/0005181404?sid=101', 'https://n.news.naver.com/article/newspaper/015/0005180364?date=20250905']

# 데이터 로드
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

# 데이터 청킹
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 랭체인 허브에 있는 높은 순위의 rag 프롬프트를 가져다 사용한다.
prompt = hub.pull("rlm/rag-prompt")
prompt

llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(
    "삼성전자의 ai 관련 사업 계획에 대해 알려줘"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

문서의 수: 3


'삼성전자는 자체 AI 모델 ‘가우스’를 고도화하는 한편 구글 제미나이 등 파트너 모델을 함께 활용하는 ‘하이브리드 AI’ 전략으로, 온디바이스 AI를 기본으로 필요할 때 클라우드 AI를 결합하는 방식을 추진합니다. 제품 측면에서는 연내 4억 대 이상의 갤럭시 기기에 AI 기능을 탑재하고 프리미엄에서 보급형으로 빠르게 확대해 대중화하겠다고 했습니다. 조직 차원에선 2030년까지 전 업무 영역의 약 90%에 AI를 적용해 AI가 현장에서 판단·결정하는 ‘AI 선도 컴퍼니’로 전환하겠다는 목표를 세웠습니다.'